In [2]:
from esper.supercuts import *
from query.models import Shot
from rekall.temporal_predicates import overlaps

In [3]:
from rekall.video_interval_collection import VideoIntervalCollection
from rekall.interval_list import IntervalList
from rekall.merge_ops import payload_plus
from rekall.payload_predicates import payload_satisfies
from rekall.temporal_predicates import overlaps
from rekall.parsers import in_array, merge_dict_parsers, bbox_payload_parser, dict_payload_parser
from esper.caption_metadata import caption_metadata_for_video
from esper.captions import get_all_segments
from esper.prelude import esper_widget
import time
from IPython.display import display

Loading the document list and lexicon
Matched 571 documents to videos
0 documents have no videos
71 videos have no documents


# Query

In [22]:
def show_query(video_id, text):
    videos = Video.objects.filter(id__in=[video_id]).all()
    
#     # Annotate face rows with start and end frames and the video ID
#     faces_with_character_actor_qs = FaceCharacterActor.objects.annotate(
#         min_frame=F('face__frame__number'),
#         max_frame=F('face__frame__number'),
#         video_id=F('face__frame__video_id'),
#         character_name=F('characteractor__character__name')
#     ).filter(video_id__in=[v.id for v in videos])

#     frames_with_identity = VideoIntervalCollection.from_django_qs(
#         faces_with_character_actor_qs,
#         with_payload=in_array(
#             dict_payload_parser(VideoIntervalCollection.django_accessor, { 'character': 'character_name' }),
#         )
#     ).coalesce(payload_merge_op=payload_plus)
    
    def convert_to_frames(intrvllist_seconds, fps):
        return intrvllist_seconds.map(
            lambda intrvl: (int(intrvl.start * fps), int(intrvl.end * fps), intrvl.payload)
        )
    
    all_segments = get_all_segments([video.id for video in videos])
    captions_interval_collection = VideoIntervalCollection({
        video: convert_to_frames(IntervalList(intervals), Video.objects.get(id=video).fps)
        for video, intervals in all_segments
    }).dilate(24).coalesce(payload_merge_op = payload_plus).map(
        lambda intrvl: (intrvl.start, intrvl.end, ' '.join(intrvl.payload))
    )
    
#     frames_with_antagonist = frames_with_identity.filter(
#         lambda intrvl: any(antagonist_name in char['character'] for char in intrvl.payload)
#     )
    
#     frames_with_antagonist_and_text = captions_interval_collection.filter_against(
#         frames_with_antagonist, predicate=overlaps(), working_window=10).coalesce().filter_length(min_length=48)
    
    statements_with_questions = captions_interval_collection.filter(
        lambda intrvl: text in intrvl.payload.lower()
    )
    
    result = intrvllists_to_result_with_objects(statements_with_questions, lambda a, b: [])
    widget = esper_widget(result, jupyter_keybindings=True, disable_captions=True)
    display(widget)
    return widget, result

In [33]:
def convert_segments(segments):
    output_segments = []
    for seg in segments:
        for res in result['result']:
            if res['elements'][0]['min_frame'] == seg[0]:
                output_segments.append((res['elements'][0]['min_frame'], res['elements'][0]['max_frame']))
    print(output_segments)

In [36]:
widget, result = show_query(28, 'why')

VGridWidget(jsglobals={'queries': [['Reaction shots in Apollo 13 (rekall)', 'def reaction_shots_apollo_13():\n…

In [19]:
selected_segments_braveheart = [
    (result['result'][i]['elements'][0]['min_frame'], result['result'][i]['elements'][0]['min_frame'])
    for i in widget.selected
]
print(selected_segments_braveheart)

[(136941, 136941), (138760, 138760), (214455, 214455)]


In [37]:
convert_segments(selected_segments_braveheart)

[(136941, 137580), (138760, 139111), (214455, 214519)]


# Revenge of the Sith

In [24]:
start = time.time()

In [34]:
widget, result = show_query(186, 'how')

VGridWidget(jsglobals={'queries': [['Reaction shots in Apollo 13 (rekall)', 'def reaction_shots_apollo_13():\n…

In [30]:
selected_segments_rots = [
    (result['result'][i]['elements'][0]['min_frame'], result['result'][i]['elements'][0]['min_frame'])
    for i in widget.selected
]
print(selected_segments_rots)

[(53622, 53622), (53780, 53780)]


In [35]:
convert_segments(selected_segments_rots)

[(53622, 53772), (53780, 53973)]


In [31]:
end = time.time()

In [32]:
print("Seconds to label: ", end - start)

Seconds to label:  160.84781956672668


# Steve Jobs

In [135]:
start = time.time()

In [137]:
widget, result = show_query(520, 'sculley')

VGridWidget(jsglobals={'queries': [['Reaction shots in Apollo 13 (rekall)', 'def reaction_shots_apollo_13():\n…

In [138]:
selected_segments_jobs = [
    (result['result'][i]['elements'][0]['min_frame'], result['result'][i]['elements'][0]['min_frame'])
    for i in widget.selected
]
print(selected_segments_jobs)

[(45475, 45475), (95252, 95252)]


In [139]:
end = time.time()

In [140]:
print("Seconds to label: ", end - start)

Seconds to label:  298.2629656791687


# Guardians of the Galaxy

In [144]:
start = time.time()

In [145]:
widget, result = show_query(74, 'ronan')

VGridWidget(jsglobals={'queries': [['Reaction shots in Apollo 13 (rekall)', 'def reaction_shots_apollo_13():\n…

In [146]:
selected_segments_gotg = [
    (result['result'][i]['elements'][0]['min_frame'], result['result'][i]['elements'][0]['min_frame'])
    for i in widget.selected
]
print(selected_segments_gotg)

[(92487, 92487), (103646, 103646)]


In [147]:
end = time.time()

In [148]:
print("Seconds to label: ", end - start)

Seconds to label:  112.39082598686218


# Daddy's Home

In [149]:
start = time.time()

In [150]:
widget, result = show_query(334, 'dusty')

VGridWidget(jsglobals={'queries': [['Reaction shots in Apollo 13 (rekall)', 'def reaction_shots_apollo_13():\n…

In [151]:
selected_segments_daddy = [
    (result['result'][i]['elements'][0]['min_frame'], result['result'][i]['elements'][0]['min_frame'])
    for i in widget.selected
]
print(selected_segments_daddy)

[(30813, 30813), (31160, 31160), (118618, 118618)]


In [152]:
end = time.time()

In [153]:
print("Seconds to label: ", end - start)

Seconds to label:  272.71405601501465


# Batman v superman: no caption file

In [178]:
def show_query_no_captions(video_id, antagonist_name):
# video_id=299
# antagonist_name="luthor"
# if True:
    videos = Video.objects.filter(id__in=[video_id]).all()
    
    # Annotate face rows with start and end frames and the video ID
    faces_with_character_actor_qs = FaceCharacterActor.objects.annotate(
        min_frame=F('face__frame__number'),
        max_frame=F('face__frame__number'),
        video_id=F('face__frame__video_id'),
        character_name=F('characteractor__character__name')
    ).filter(video_id__in=[v.id for v in videos])

    frames_with_identity = VideoIntervalCollection.from_django_qs(
        faces_with_character_actor_qs,
        with_payload=in_array(
            dict_payload_parser(VideoIntervalCollection.django_accessor, { 'character': 'character_name' }),
        )
    ).coalesce(payload_merge_op=payload_plus)
    
    def convert_to_frames(intrvllist_seconds, fps):
        return intrvllist_seconds.map(
            lambda intrvl: (int(intrvl.start * fps), int(intrvl.end * fps), intrvl.payload)
        )
    
    all_segments = get_all_segments([video.id for video in videos])
    captions_interval_collection = VideoIntervalCollection({
        video: convert_to_frames(IntervalList(intervals), Video.objects.get(id=video).fps)
        for video, intervals in all_segments
    }).dilate(10).coalesce(payload_merge_op = payload_plus).dilate(-10).map(
        lambda intrvl: (intrvl.start, intrvl.end, ' '.join(intrvl.payload))
    )
    
    frames_with_antagonist = frames_with_identity.filter(
        lambda intrvl: any(antagonist_name in char['character'] for char in intrvl.payload)
    )
    
    #print(frames_with_antagonist.get_intervallist(299))
    
    frames_with_antagonist_and_text = captions_interval_collection.filter_against(
        frames_with_antagonist, predicate=overlaps(), working_window=10).coalesce().filter_length(min_length=48)
    
    result = intrvllists_to_result_with_objects(
        frames_with_antagonist.dilate(10).coalesce().dilate(-10).filter_length(
            min_length=48
        ), lambda a, b: [])
    widget = esper_widget(result, jupyter_keybindings=True)
    display(widget)
    return widget, result

In [179]:
start = time.time()

In [180]:
widget, result = show_query_no_captions(299, 'luthor')

No document for video id: 299


VGridWidget(jsglobals={'queries': [['Reaction shots in Apollo 13 (rekall)', 'def reaction_shots_apollo_13():\n…

In [181]:
selected_segments_bvs = [
    (result['result'][i]['elements'][0]['min_frame'], result['result'][i]['elements'][0]['min_frame'])
    for i in widget.selected
]
print(selected_segments_bvs)

[(60638, 60638), (73664, 73664), (167760, 167760), (233131, 233131), (248916, 248916)]


In [182]:
end = time.time()

In [183]:
print("Seconds to label: ", end - start)

Seconds to label:  456.8938891887665
